In [ ]:
import numpy as np

# Parameters
rho = 10  # ρ
d = 10  # Dimensionality of the subspace
n = 50  # Ambient space dimensionality
L = 100  # Number of subspaces
Nl = rho * d  # Number of points per subspace
N = L * Nl  # Total number of points


def gram_schmidt(vectors):
    num_vectors = vectors.shape[0]
    orthogonalized_vectors = np.zeros_like(vectors, dtype=np.float64)

    for i in range(num_vectors):
        # Start with the original vector
        vec = vectors[i].copy()
        for j in range(i):
            # Subtract the projection onto each previously orthogonalized vector
            proj = np.dot(orthogonalized_vectors[j], vec) / np.dot(orthogonalized_vectors[j], orthogonalized_vectors[j])
            vec -= proj * orthogonalized_vectors[j]
        # Store the orthogonalized vector
        orthogonalized_vectors[i] = vec

    # Normalize vectors and handle zero-norm cases
    for i in range(num_vectors):
        norm = np.linalg.norm(orthogonalized_vectors[i])
        if norm > 0:
            orthogonalized_vectors[i] /= norm

    return orthogonalized_vectors

X = []
# Loop through each subspace
start_index = 0
for l in range(L):
    # Sample a Gaussian matrix A and obtain U using Schmidt orthogonalization
    A = np.random.randn(n, d)
    # U, _ = np.linalg.qr(A)  # U is orthogonal
    U = gram_schmidt(A)
    # Sample a Gaussian matrix B and normalize columns to lie on the unit sphere
    B = np.random.randn(d, Nl)
    B = B / np.linalg.norm(B, axis=0, keepdims=True)  # Normalize columns
    
    # Construct data for the l-th subspace
    X_l = U @ B
    
    # Insert X_l into the global data matrix
    end_index = start_index + Nl
    # X[:, start_index:end_index] = X_l
    start_index = end_index
    X.append(X_l)

X = np.concatenate(X, axis=1)
print("Generated data matrix X with shape:", X.shape)
print(N)

Generated data matrix X with shape: (50, 10000)
10000


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pylops
import pyproximal

# Parameters
epsilon = 1e-5
threshold = 0.0005 # For calculating support size
support_sizes = []

# Loop through each column of X
for i in range(10):
    # Remove the i-th column from X to get X(-i)
    X_minus_i = np.delete(X, i, axis=1)
    X_i = X[:, i]

    # Operator
    Aop = pylops.MatrixMult(X_minus_i)
    Aop.explicit = False  # Temporary solution whilst PyLops gets updated

    # Observed data
    y = X_i

    # Define the optimization problem
    f = pyproximal.AffineSet(Aop, y, niter=20)  # Constraint ||X_i - X_minus_i @ β||_2 <= epsilon
    g = pyproximal.L1()  # Regularization term ||β||_1

    # Solve the optimization problem using ADMM
    beta_early = pyproximal.optimization.primal.ADMM(f, g, np.zeros(X_minus_i.shape[1]), 0.1, niter=10, show=False)[0]
    beta = pyproximal.optimization.primal.ADMM(f, g, np.zeros(X_minus_i.shape[1]), 0.1, niter=150, show=False)[0]

    # Calculate the support size for the final solution
    support_size = np.sum(np.abs(beta) > threshold)
    support_sizes.append(support_size)

# Output the minimum and maximum support sizes
print("Minimum support size:", min(support_sizes))
print("Maximum support size:", max(support_sizes))

Minimum support size: 11
Maximum support size: 75
